Carlos Sevilla Silva A00826925

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar el juego de la vida usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [1]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 5.0 MB/s 
     |████████████████████████████████| 60 kB 3.2 MB/s 


In [2]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import SingleGrid, MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random

## Crear el modelo

Antes que nada el presente modelo se encuentra basado en el [tutorial introductorio](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html). Lo modifiqué un poco para que funcionara para el presente problema pero en esencia es lo mismo.

In [3]:
tiempoEjec = 0
movRobots = 0

def get_grid(model):

    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
          if isinstance(content, Robot):
            grid[x][y] = 6
          else:
            grid[x][y] = content.live
        
    return grid
  
class RandomMovement(Agent):
    grid = None
    x = None
    y = None
    moore = True

    def __init__(self, unique_id, pos, model, moore=True):
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, self.moore, True)
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)


class Box(Agent):

    def __init__(self, unique_id, pos, model):

        super().__init__(unique_id, model)

        self.live = 1
        self.next_state = self.live
        

    def step(self):
        x, y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])     

    def advance(self):
        self.live = self.next_state
        

class Robot(RandomMovement):
    
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)
        self.live = 6
        self.boxes = 0
        self.boxesNext = 0
        self.towerX = -1
        self.towerY = -1
        self.move = True
        

    def step(self):
        global tiempoEjec
        self.move = True
        # self.random_move()
        self.x,self.y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        #cellContents = [obj for obj in this_cell if isinstance(obj, Box)]
        for content in this_cell:
          if isinstance(content, Box):

            if self.boxes == 0 and (content.live == 1):
              self.boxes = 1

              content.next_state = 0

            elif self.boxes == 1 and (self.towerX == -1) and (content.live == 1):
              self.towerX = self.x 
              self.towerY = self.y

              content.next_state = content.live + 1

              self.boxes = 0
              ##
              tiempoEjec = time.time()
              ##
            elif self.boxes == 1 and (self.x == self.towerX and self.y == self.towerY):
              if content.live < 5:

                content.next_state = content.live + 1

                self.boxes = 0
                ##
                tiempoEjec = time.time()
                ##
                if content.next_state == 5:
                  self.towerX = -1
                  self.towerY = -1
              else:
                self.towerX = -1
                self.towerY = -1
            elif self.boxes == 0 and (content.live == 5):
              self.boxes = 0
            elif self.boxes == 2:
              self.towerX = -1
              self.towerY = -1
            

            
    def advance(self):
        self.boxes = self.boxes
        self.towerX = self.towerX
        self.towerY = self.towerY
        self.move = True
        global movRobots
        movRobots = movRobots + 1
        
        if self.boxes == 1 and (self.towerX != -1):
          #if already has tower spot and has a box, and is not on tower, go to it.

          if self.x < self.towerX and self.x < GRID_SIZE-1:
            #x = self.x + 1
            this_cell = self.model.grid.get_cell_list_contents([(self.x+1,self.y)])
            for content in this_cell:
              if isinstance(content,Robot):
                self.move = False
            
            if self.move:
              self.model.grid.move_agent(self,(self.x+1,self.y))

          elif self.x > self.towerX and self.x > 0:
            #x = self.x - 1
            this_cell = self.model.grid.get_cell_list_contents([(self.x-1,self.y)])
            for content in this_cell:
              if isinstance(content,Robot):
                self.move = False
            
            if self.move:
              self.model.grid.move_agent(self,(self.x-1,self.y))

          elif self.y < self.towerY and self.y < GRID_SIZE-1:
            #y = self.y + 1
            this_cell = self.model.grid.get_cell_list_contents([(self.x,self.y+1)])
            for content in this_cell:
              if isinstance(content,Robot):
                self.move = False
            
            if self.move:
              self.model.grid.move_agent(self,(self.x,self.y+1))

          elif self.y > 0:
            #y = self.y - 1
            this_cell = self.model.grid.get_cell_list_contents([(self.x,self.y-1)])
            for content in this_cell:
              if isinstance(content,Robot):
                self.move = False
            
            if self.move:
              self.model.grid.move_agent(self,(self.x,self.y-1))
        else:
          #if doesnt have a box, or if you have a box and not a tower, move randomly to find a new box
          self.randomMove = random.randint(1, 4)

          if self.randomMove == 1 and self.x < GRID_SIZE-1:
            #x = self.x + 1
            this_cell = self.model.grid.get_cell_list_contents([(self.x+1,self.y)])
            for content in this_cell:
              if isinstance(content,Robot):
                self.move = False
            
            if self.move:
              self.model.grid.move_agent(self,(self.x+1,self.y))

          elif self.randomMove == 2 and self.x > 0:
            #x = self.x - 1
            this_cell = self.model.grid.get_cell_list_contents([(self.x-1,self.y)])
            for content in this_cell:
              if isinstance(content,Robot):
                self.move = False
            
            if self.move:
              self.model.grid.move_agent(self,(self.x-1,self.y))

          elif self.randomMove == 3 and self.y < GRID_SIZE-1:
            #y = self.y + 1
            this_cell = self.model.grid.get_cell_list_contents([(self.x,self.y+1)])
            for content in this_cell:
              if isinstance(content,Robot):
                self.move = False
            
            if self.move:
              self.model.grid.move_agent(self,(self.x,self.y+1))

          elif self.y > 0:
            #y = self.y - 1
            this_cell = self.model.grid.get_cell_list_contents([(self.x,self.y-1)])
            for content in this_cell:
              if isinstance(content,Robot):
                self.move = False
            
            if self.move:
              self.model.grid.move_agent(self,(self.x,self.y-1))
       
class GameLifeModel(Model):
    '''
    Define el modelo del juego de la vida.
    '''
    def __init__(self, width, height, boxes, robots):
        self.num_agents = width * height * boxes
        self.grid = MultiGrid(width, height, False)
        self.schedule = SimultaneousActivation(self)
        arr = [[0 for i in range(height)] for j in range(width)]
        print(arr)

        i=0
        while i < boxes:
          x = self.random.randrange(width)
          y = self.random.randrange(height)
#          print(arr[x][y])
          if arr[x][y] == 0:
#            print('im in')
            box = Box((x+np.random.randint(0,999999),y+np.random.randint(0,999999)),(x,y),self)
            self.grid.place_agent(box, (x, y))
            self.schedule.add(box)
            arr[x][y] = 1
            i = i+1


        i=0
        while i < robots:
          x = self.random.randrange(width)
          y = self.random.randrange(height)
#          print(arr[x][y])
          if arr[x][y] == 0:
#            print('im in robot')
            robot = Robot((x+np.random.randint(0,999999),y+np.random.randint(0,999999)),(x,y),self)
            self.grid.place_agent(robot, (x, y))
            self.schedule.add(robot)
            arr[x][y] = 1
            i = i+1


#        x = self.random.randrange(width)
#        y = self.random.randrange(height)
#        box = Box((x+np.random.randint(0,999999),y+np.random.randint(0,999999)),(x,y),self)
#        self.grid.place_agent(box, (x, y))
#        self.schedule.add(box)



#        x = self.random.randrange(width)
#        y = self.random.randrange(height)
#        robot = Robot((x+np.random.randint(0,999999),y+np.random.randint(0,999999)),(x,y),self)
#        self.grid.place_agent(robot, (x, y))
#        self.schedule.add(robot)

        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

A continuación corremos el modelo

In [4]:
# Definimos el tamaño del Grid

#Carlos Sevilla Silva
#A000826925


GRID_SIZE = 10
boxes = 30
robots = 5
# Definimos el número de generaciones a correr
NUM_GENERATIONS = 300
firstMention = 0
movTotales = 0
segundos = 10


# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
time_end = time.time() + segundos
model = GameLifeModel(GRID_SIZE, GRID_SIZE, boxes, robots)
while time.time() < time_end:
  model.step()
#for i in range(NUM_GENERATIONS):
#  model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

print('movimientos totales de robots: ', movRobots)
print('tiempo de ejecucion al apilar la ultima caja en la simulacion: ', str(datetime.timedelta(seconds=(tiempoEjec))))
#print('movimientos totales: ', movTotales*aspiradoras)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Tiempo de ejecución: 0:00:10.000395
movimientos totales de robots:  86580
tiempo de ejecucion al apilar la ultima caja en la simulacion:  18871 days, 14:51:50.201364


Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [5]:
all_grid = model.datacollector.get_model_vars_dataframe()

Graficamos la información usando `matplotlib`

In [6]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [7]:
anim